# TODO: Improvements based on lec2
## Key Points
1. **Initialization matters**: wants to logits to be close to 0 at the very beginning to avoid large losses.
2. **Saturation of the tanh function**: break the gradient flow. plot tanh intermediate values to see how it saturates.
   1. Gradient always deceases as we go back through tanh.
   2. Dead neurons: tanh (or sigmoid or relu) satrurates and cause the gradient to be 0 for all instances.
   3. Initialization: motivation is to keep the distribution's variance of the input and output of each layer the same. If no activation, can multiply by sqrt of (1 / num_of_inputs).
   4. Kaiming's initialization: for relu, multiply by sqrt of (2 / number of inputs), where 2 is called "gain", used to compensate for the fact that relu is not symmetric around 0. "gain" has different values for different activation functions (see pytorch docs aboud weight initialization).
   5. "Gain" is important: if no activation, then using gain=1 is enough to keep the variance the same, but with squashing activation functions like tanh, the variance will be smaller and smaller, so we need to multiply by a larger value to keep the variance the same.
   6. Modern techniques like Adam, BN, residual connections, make initialization less important, but still useful.
3. **BN motivation**: make the outputs before the activation function subject to a Gaussian (if too small, tanh has no effect; if too large, tanh saturates)
   1. It's a direct idea: just normalize it to have mean 0 and variance 1.
   2. Then scale and shift using learnable parameters.
   3. BN's side effect: the activation value of an instance is dependent on the other instances in the batch (which is randmomly sampled), thus cause jittering in the training process. But this turns out to be a good thing, as it acts as a regularizer/data augmentation.
   4. BN in inference: use the mean and variance of the entire training set or maintain a running mean and variance during training.
   5. Bias is no more needed in the linear layer, as BN has its own bias (mathematically, the bias term will be substrated out by the mean of BN, so it never learns).
   6. Momentum in BN: in running mean and variance, use momentum to update the mean and variance. (small batch size can cause large variance in estimation, so use a smaller momentum value to smooth it out)
## Implementation detail
1. Torchify the code of Linear, BatchNorm1d and Tanh, which should have `__call__()` and `parameters()` methods.
2. Write: build MLP, enable the grad tracking, initialize the weights, training process.
3. No BN, run for only 1 epoch, visualize (can try some other "gain" values to see the effect):
   1. Value distribution of activation.
   2. Grad distribution of activation.
   3. Weight distribution.
   4. Updata:data ratio over time (to see the speed of training of each layer).
4. Add BN and check the robustness.
5. Other details:
   1. The position of the BN.
   2. Make the last layer less confident.

In [1]:
import torch
import torch.nn.functional as F
from matplotlib import pyplot as plt
import random

# Data preparation

In [2]:
words = open("../names.txt", "r").read().splitlines()
chars = sorted(set('.'.join(words)))
stoi = {char:i for i, char in enumerate(chars)}
itos = {i:char for i, char in enumerate(chars)}

In [4]:
ratio1 = 0.8
ratio2 = 0.9
chunk = 3

def create_data(ws, chunk=3):
    X = []
    Y = []
    
    for word in ws:
        Xt = [0] * chunk # Cool
        for ch in word + '.': # Dont forget to add ending token.
            X.append(Xt)
            Y.append(stoi[ch])   
            Xt = Xt[1:] + [stoi[ch]]

    return torch.tensor(X), torch.tensor(Y)

random.shuffle(words) # No obvious affect (original words already shuffled)
n1 = int(ratio1 * len(words))
n2 = int(ratio2 * len(words))
Xt, Yt = create_data(words[:n1], chunk=chunk)
Xdev, Ydev = create_data(words[n1:n2], chunk=chunk)
Xte, Yte = create_data(words[n2:], chunk=chunk)
print(Xt.shape)
print(Xdev.shape)

torch.Size([182578, 3])
torch.Size([22777, 3])


# Construct the component classes

In [12]:
class Linear:
    def __init__(self, n_in, n_out, bias=True):
        self.W = torch.randn((n_in, n_out)) * (n_in)**(-0.5)
        self.b = torch.zeros(n_out) if bias else None # zeros
        self.out = None # For visualization

    def __call__(self, x):
        self.out = x @ self.W
        if self.b is not None:
            self.out += self.b
        return self.out   

    def parameters(self):
        return [self.W, self.b] if self.b is not None else [self.W]

test_linear = Linear(10, 50)
x_test = torch.randn((32, 10))
print(test_linear(x_test).shape)
print(test_linear.parameters()[0].shape)

torch.Size([32, 50])
torch.Size([10, 50])


In [ ]:
class BN1d:
    def __init__(self, num_feat, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum

        self.gamma = torch.ones(num_feat) # ones
        self.beta = torch.zeros(num_feat) # zeros

        self.training = True
        self.rmean = torch.ones(num_feat) 
        self.rvar = torch.zeros(num_feat)

        self.out = None

    def __call__(self, x):
        if self.training:
            meani = torch.mean(x, dim=0, keepdim=True) 
            vari = torch.var(x, dim=0, keepdim=True)
        else:
            meani = self.rmean
            vari = self.rvar
        
        if not self.training:
            self.rmean = (1- self.momentum) * self.rmean + self.momentum * meani 
            self.rvar = (1 - self.momentum) * self.rvar + self.momentum * vari
        
        self.out = self.gamma * (x - meani) / torch.sqrt(vari + self.eps) + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

BN_test = BN1d(4)
x_test = torch.randn((5, 4))
print(BN_test(x_test))
print((x_test - x_test.mean(0, keepdim=True))/(torch.sqrt(x_test.var(0, keepdim=True) + 1e-5)))

tensor([[-1.3652,  1.3443, -0.7855,  1.2520],
        [ 1.1592, -1.4120, -0.0362, -0.6560],
        [ 0.0312, -0.0277,  1.0200, -0.8850],
        [-0.5356,  0.3590,  0.9782, -0.6227],
        [ 0.7104, -0.2636, -1.1766,  0.9117]])
tensor([[-1.3652,  1.3443, -0.7855,  1.2520],
        [ 1.1592, -1.4120, -0.0362, -0.6560],
        [ 0.0312, -0.0277,  1.0200, -0.8850],
        [-0.5356,  0.3590,  0.9782, -0.6227],
        [ 0.7104, -0.2636, -1.1766,  0.9117]])


In [5]:
X_ = F.one_hot(Xt, num_classes=27).float()
print(X_.shape)
W = torch.randn(27, 3)
print((X_ @ W).shape)

torch.Size([182578, 3, 27])
torch.Size([182578, 3, 3])


In [ ]:
embedding_size = 3

def get_embedding(embedding_size):
    return torch.randn(27, embedding_size, requires_grad=True)

In [ ]:
def get_MLP(chunk, embedding_size, num_neurons):
    Ws = []
    bs = []

    # Input layer
    W = torch.randn(chunk * embedding_size, num_neurons[0], requires_grad=True)
    b = torch.randn(num_neurons[0])
    Ws.append(W)
    bs.append(b)

    # Hidden layer
    if len(num_neurons) > 1:
        for n1, n2 in zip(num_neurons, num_neurons[1:]):
            W = torch.randn(n1, n2, requires_grad=True)
            b = torch.randn(n2, requires_grad=True)
            Ws.append(W)
            bs.append(b)

    # Output layer
    W = torch.randn(num_neurons[-1], 27, requires_grad=True)
    b = torch.randn(27)
    Ws.append(W)
    bs.append(b)

    return Ws, bs

W, b = get_MLP(chunk, embedding_size, [100])
print(W[0].shape)

# Train & val scripts

In [ ]:
def go_through(xx, embed, Ws, bs):
    # Through embedding
    xx = F.one_hot(xx, num_classes=27).float()
    xx = (xx @ embed).view(xx.shape[0], -1)
    xx = F.tanh(xx) # Easy to forget

    # Through MLP
    num_layers = len(Ws)
    for i, (W, b) in enumerate(zip(Ws, bs)):
        xx = xx @ W + b
        if i < num_layers - 1: # No activation after the last layer
            xx = F.tanh(xx)
    return xx

In [ ]:
def train_MLP(X, Y, embed, Ws, bs, batch_size, epochs, lr):
    loss_list = []
    for epoch in range(epochs):
        # Zero out the grad (easy to forget)
        embed.grad = None
        for W, b in zip(Ws, bs):
            W.grad = None
            b.grad = None
        
        # Select batch
        indices = torch.randint(low=0, high=X.shape[0], size=(batch_size,))
        xx = X[indices]
        yy = Y[indices]

        xx = go_through(xx, embed, Ws, bs)
        
        # Compute loss
        loss = F.cross_entropy(xx, yy)
        loss.backward()
        loss_list.append(loss.item())

        # Update params (Use tensor.data, because leaf tensor cannot do in-place operations)
        embed.data += -lr * embed.grad
        for W, b in zip(Ws, bs):
            W.data += -lr * W.grad
            if b.requires_grad == True:
                b.data += -lr * b.grad
            
    return loss_list

# embed = get_embedding(embedding_size)
# embed.requires_grad = True
# Ws, bs = get_MLP(chunk, embedding_size, [100])
# loss_list = train_MLP(Xt, Yt, embed, Ws, bs, 64, 10000, 0.1)
# plt.plot(loss_list)

In [ ]:
def dev_MLP(X, Y, embed, Ws, bs):
    xx = go_through(X, embed, Ws, bs)
    loss = F.cross_entropy(xx, Y)
    print(f"loss: {loss}")

# Train & Eval the MLP

In [ ]:
chunk = 3
embedding_size = 32
num_neurons = [128, 256, 128]

embed = get_embedding(embedding_size)
Ws, bs = get_MLP(chunk, embedding_size, num_neurons)

for W, b in zip(Ws, bs):
    print(W.shape)

batch_size = 32
epochs = 100000
lr = 0.1

In [ ]:
loss_list = train_MLP(Xt, Yt, embed, Ws, bs, batch_size, 50000, 0.01)
plt.plot(loss_list)

In [ ]:
dev_MLP(Xdev, Ydev, embed, Ws, bs)

# Infer

In [64]:
def infer_names(num, embed, Ws, bs): # Cant infer in parallel
    for i in range(num):
        result = []
        xx = [0] * chunk
        while True:
            logits = go_through(torch.tensor(xx).unsqueeze(0), embed, Ws, bs)
            probs = logits.exp() / logits.exp().sum(dim=1)
            next_token = torch.multinomial(probs[0], num_samples=1)
            result.append(next_token.item())
            xx = xx[1:] + [next_token]
            if next_token == 0:
                break
        result = [itos[char] for char in result]
        print(''.join(result))
    
infer_names(2, embed, Ws, bs)

jip.
luncekealie.
